In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    Correspond to an offline power prediction model to be used to predict the mean 
    power consumption, over the execution interval, of a job submitted on the Erurora HPC. 
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
interval_comment = "Alina"
#start_time = pd.to_datetime('2014-06-30')
tart_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')
train_start_time = pd.to_datetime('2014-06-30')
train_end_time = pd.to_datetime('2014-10-01')
test_end_time = pd.to_datetime('2014-11-01')


infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
#infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow_64_359350"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [2]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(jobs.shape[0], interval_comment))

# clean user names
jobs['user'] = jobs.apply(lambda row: row['user'].split('@')[0], axis=1)

jobs_to_nodes_whole_data contains 469095 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 336787 long jobs in the Alina's interval


In [5]:
jobs_not_zero_quality = jobs[jobs['real_pow'] > 0]

jobs_test = jobs_not_zero_quality[pd.to_datetime(jobs_not_zero_quality['end_time']) <= test_end_time]
jobs_test = jobs_test[pd.to_datetime(jobs_test['run_start_time']) > train_end_time]
print("test set contains {} records".format(jobs_test.shape[0]))

test set contains 24918 records


In [6]:
#jobs_test = jobs_test[pd.to_datetime(jobs_test['end_time']) - pd.to_datetime(jobs_test['run_start_time']) > np.timedelta64(5, 's')]
#jobs_test.shape[0]

In [7]:
### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(jobs_test, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

after the clean, jobs_to_nodes_whole_data contains 444610 records
merged_jobs_to_nodes contains 25569 records


In [8]:
infile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
df_user_job_node = pd.read_csv(infile_user_job_node + ".csv", index_col=0)
#infile_user_job = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_whole"
#df_user_job = pd.read_csv(infile_user_job + ".csv", index_col=0)
#infile_user_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_node_whole"
#df_user_node = pd.read_csv(infile_user_node + ".csv", index_col=0)
#infile_user = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_whole"
#df_user = pd.read_csv(infile_user + ".csv", index_col=0)
#infile_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_node_whole"
#df_node = pd.read_csv(infile_node + ".csv", index_col=0)

df_user_job_node_copy = df_user_job_node

In [10]:
#debug
df_user_job_node_copy.shape[0]

23260

In [11]:
df_user_job_node.head(10)

,user,job_name,node_id,count,mean_core
0,a07smr01,A2REPO,7,1,12.412156
1,a07smr01,A2REPO,33,1,3.937672
2,a07smr01,A2REPO,35,59,4.375295
3,a07smr01,A2REPO,46,24,2.406068
4,a07smr01,ARCHIVE,7,1,11.895912
5,a07smr01,ARKI,33,7,5.766090
6,a07smr01,ARKI,35,81,4.299856
7,a07smr01,ARKI,46,27,3.992453
8,a07smr01,INIT,33,4,4.631090
9,a07smr01,INIT,35,78,3.525155


### some study of jobs distribution

In [12]:
grouped_jobs_test = merged_jobs_to_nodes.groupby(['user', 'job_name', 'node_id']).size().reset_index(name='counts')

print("{}/{} distinct execution".format(grouped_jobs_test.shape[0], merged_jobs_to_nodes.shape[0]))

22114/25569 distinct execution


In [13]:
grouped_jobs_test = jobs_test.groupby(['user', 'job_name']).size().reset_index(name='counts')
print("{}/{} distinct job names".format(grouped_jobs_test.shape[0], merged_jobs_to_nodes.shape[0]))

7669/25569 distinct job names


In [14]:
grouped_jobs_test.sort_values('counts')

,user,job_name,counts
5680,asaetti0,run6082.sh,1
5770,asaetti0,run6163.sh,1
5771,asaetti0,run6164.sh,1
5772,asaetti0,run6165.sh,1
5773,asaetti0,run6166.sh,1
5774,asaetti0,run6167.sh,1
5775,asaetti0,run6168.sh,1
5776,asaetti0,run6169.sh,1
5778,asaetti0,run6170.sh,1
5779,asaetti0,run6171.sh,1


In [17]:
convlwalle = jobs_test[jobs_test['job_name'] == 'convlwalle'][['real_pow','cpu_req', 'job_name']]
convlwalle['mean_core'] = convlwalle['real_pow'] / convlwalle['cpu_req']
convlwalle

,real_pow,cpu_req,job_name,mean_core
job_id,,,,
309425,85.847301,2,convlwalle,42.923650
309577,321.243457,7,convlwalle,45.891922
309592,205.684218,8,convlwalle,25.710527
309597,182.619073,8,convlwalle,22.827384
309598,298.175755,8,convlwalle,37.271969
309600,192.541710,8,convlwalle,24.067714
309605,314.478921,7,convlwalle,44.925560
309613,491.209318,12,convlwalle,40.934110
309615,255.390272,7,convlwalle,36.484325


In [117]:
''' 1 dataframe
Predict the power consumption of a job.
    If the user and the job name are known, than it use the method 1 [user,job,nodes]
    if only the user is known, than it use the method 2 [user,nodes]
    Otherwise it use the method 3 [nodes]
'''

def predict_pow(user, job_name, used_nodes, used_cpus):
    pred_pow = 0.0
    
    methods = [0, 0, 0, 0, 0]
    
    # check if the method 1 is available
    mean_core_user_job = df_user_job_node.loc[(df_user_job_node['user']==user) & (df_user_job_node['job_name'] == job_name)]
    if(mean_core_user_job.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
        
        #mean_core_wholeNodes = mean_core_user_job['mean_core'].mean()
        mean_core_wholeNodes = np.average(mean_core_user_job['mean_core'], weights=mean_core_user_job['count'])
        
        for node, ncpu in zip(used_nodes, used_cpus):
            mean_node = mean_core_user_job.loc[(mean_core_user_job['node_id']==node)]
            if(mean_node.shape[0] != 0):
                #pred_pow += mean_node['mean_core'].iloc[0] * ncpu
                pred_pow += mean_node['mean_core'].mean() * ncpu
                methods[0] += 1
            else:
                pred_pow += mean_core_wholeNodes * ncpu
                methods[1] += 1
        return (methods, pred_pow)
    
    
    # check if the method 2 is available
    mean_core_user = df_user_job_node.loc[df_user_job_node['user']==user]
    if(mean_core_user.shape[0] != 0):  # if != 0 than user exists on hystoric data
        
        #mean_core_wholeUserJobs = mean_core_user['mean_core'].mean()
        mean_core_wholeUserJobs = np.average(mean_core_user['mean_core'], weights=mean_core_user['count'])
        
        for node, ncpu in zip(used_nodes, used_cpus):
            mean_node = mean_core_user.loc[(mean_core_user['node_id']==node)]
            if(mean_node.shape[0] != 0):
                #pred_pow += mean_node['mean_core'].iloc[0] * ncpu
                pred_pow += mean_node['mean_core'].mean() * ncpu
                methods[2] += 1
            else:
                pred_pow += mean_core_wholeUserJobs * ncpu
                methods[3] += 1
        return (methods, pred_pow)
    
    
    # method 3 is the only one available
    for node, ncpu in zip(used_nodes, used_cpus):
        mean_node = df_user_job_node.loc[df_user_job_node['node_id']==node]
        mean_core_node = np.average(mean_node['mean_core'], weights=mean_node['count'])
        
        pred_pow += mean_core_node * ncpu
        methods[4] += 1
    return (methods, pred_pow)

In [119]:
## OFFLINE 1 FILE
df_user_job_node = df_user_job_node_copy

i = 0
jobs_test['predict_off_whole'] = 0.0
#jobs_test['method_off_whole'] = 0
global_methods = [0, 0, 0, 0, 0]

for index_job_test, row_job_test in jobs_test.iterrows():
    merged_job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == row_job_test['job_id_string']]
    grouped_job_to_nodes = merged_job_to_nodes.groupby(['node_id', 'ncpus','ngpus', 'nmics']).size().reset_index(name='counts')
    
    used_nodes = grouped_job_to_nodes['node_id']
    used_cpus = grouped_job_to_nodes['ncpus']
    
    methods, predicted_pow = predict_pow(row_job_test['user'], row_job_test['job_name'], used_nodes, used_cpus)
    
    for j in range(5):
        global_methods[j] += methods[j]
    
    #print("{}: {} ------- {}".format(method, predicted_pow, row_job_test['real_pow']))
    jobs_test.at[index_job_test, 'predict_off_whole'] = predicted_pow
    #jobs_test.at[index_job_test, 'method_off_whole'] = method
    
    i += 1
    if(i%1000 == 0):
        print("{}/{}".format(i, jobs_test.shape[0]))
    #print("{}/{}".format(i, jobs_test.shape[0]))

df_user_job_node_offline = df_user_job_node

1000/8741
2000/8741
3000/8741
4000/8741
5000/8741
6000/8741
7000/8741
8000/8741


In [120]:
global_methods

[2044, 1627, 5614, 77, 29]

In [122]:
jobs_test_copy = jobs_test
#good_quality_jobs = long_jobs[long_jobs['real_pow_quality'] != 0]
#print(math.sqrt(np.var(good_quality_jobs['real_pow'])))
#good_quality_jobs = long_jobs

mean_real_pow = jobs_test['real_pow'].mean()
print("the mean is {}".format(mean_real_pow))

jobs_test['numerator'] = (jobs_test['real_pow'] - jobs_test['predict_off_whole'])**2
jobs_test['denumerator'] = (jobs_test['real_pow'] - mean_real_pow)**2
N = jobs_test['numerator'].shape[0]

r_2 = 1 - (jobs_test['numerator'].sum() / jobs_test['denumerator'].sum())
print("r_2 is {}".format(r_2))

rmse = math.sqrt(jobs_test['numerator'].sum() / N)
print("RMSE is {}".format(rmse))
print("NRMSE is {}".format(rmse/mean_real_pow))

the mean is 109.18671703238813
r_2 is 0.787867975703753
RMSE is 113.5100905031374
NRMSE is 1.039596148581579


In [31]:
#debug
df_user_job_node.shape[0]

23260

In [32]:
df_user_job_node_copy.shape[0]

23260

In [123]:
## ONLINE 1 FILE
df_user_job_node = df_user_job_node_copy
i = 0
jobs_test['predict_online_whole'] = 0.0
#jobs_test['method_off_whole'] = 0
global_methods_online = [0, 0, 0, 0, 0]

for index_job_test, row_job_test in jobs_test.iterrows():
    merged_job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == row_job_test['job_id_string']]
    grouped_job_to_nodes = merged_job_to_nodes.groupby(['node_id', 'ncpus','ngpus', 'nmics']).size().reset_index(name='counts')
    
    used_nodes = grouped_job_to_nodes['node_id']
    used_cpus = grouped_job_to_nodes['ncpus']
    
    methods, predicted_pow = predict_pow(row_job_test['user'], row_job_test['job_name'], used_nodes, used_cpus)
    
    for j in range(5):
        global_methods_online[j] += methods[j]
    
    #print("{}: {} ------- {}".format(method, predicted_pow, row_job_test['real_pow']))
    jobs_test.at[index_job_test, 'predict_online_whole'] = predicted_pow
    #jobs_test.at[index_job_test, 'method_off_whole'] = method
    
    ## ONLINE ---
    for node in used_nodes:
        job_node_mean_core = row_job_test['real_pow'] / row_job_test['cpu_req']
        #online_new_row = {'user':row_job_test['user'], 'job_name':row_job_test['job_name'], 'node_id':node, 'count':1, 'mean_core':mean_core}
        #append row to the dataframe
        
        conditions = (df_user_job_node['user'] == row_job_test['user']) & (df_user_job_node['job_name'] == row_job_test['job_name']) & (df_user_job_node['node_id'] == node)
        
        if(conditions.any()):
            ##1 avg between the previous value and the current one
            #df_user_job_node.loc[conditions, ['mean_core', 'count']] = [(df_user_job_node.loc[conditions, 'mean_core'] + mean_core) / 2, df_user_job_node.loc[conditions, 'count'] + 1]
            
            ##2 just place the current one instead the previous (bad results)
            #df_user_job_node.loc[conditions, ['mean_core', 'count']] = [mean_core, df_user_job_node.loc[conditions, 'count'] + 1]
            
            ##3 weighted avg
            prev_mean_core = df_user_job_node.loc[conditions, 'mean_core'].iloc[0]
            prev_count = df_user_job_node.loc[conditions, 'count'].iloc[0]
            df_user_job_node.loc[conditions, ['mean_core', 'count']] = [(prev_mean_core*prev_count + job_node_mean_core) / (prev_count+1), prev_count + 1]
            
        else:
            #print("-------b")
            online_new_row = {'user':row_job_test['user'], 'job_name':row_job_test['job_name'], 'node_id':node, 'count':1, 'mean_core':job_node_mean_core}
            #append row to the dataframe
            df_user_job_node = df_user_job_node.append(online_new_row, ignore_index=True)
        
        
    #df_user_job_node_online = df_user_job_node_online.groupby(['user', 'job_name', 'node_id']).agg({'mean_core':'mean'}).reset_index()
    
    
    #if(i%100 == 0):
    #    break
    
    i += 1
    if(i%1000 == 0):
        print("{}/{}".format(i, jobs_test.shape[0]))
        #break
        
    #print("{}/{}".format(i, jobs_test.shape[0]))
    
df_user_job_node_online = df_user_job_node

1000/8741
2000/8741
3000/8741
4000/8741
5000/8741
6000/8741
7000/8741
8000/8741


In [124]:
global_methods_online


[2652, 2434, 4261, 36, 8]

In [125]:
jobs_test_copy = jobs_test
#good_quality_jobs = long_jobs[long_jobs['real_pow_quality'] != 0]
#print(math.sqrt(np.var(good_quality_jobs['real_pow'])))
#good_quality_jobs = long_jobs

mean_real_pow = jobs_test['real_pow'].mean()
print("the mean is {}".format(mean_real_pow))

jobs_test['numerator'] = (jobs_test['real_pow'] - jobs_test['predict_online_whole'])**2
jobs_test['denumerator'] = (jobs_test['real_pow'] - mean_real_pow)**2
N = jobs_test['numerator'].shape[0]

r_2 = 1 - (jobs_test['numerator'].sum() / jobs_test['denumerator'].sum())
print("r_2 is {}".format(r_2))

rmse = math.sqrt(jobs_test['numerator'].sum() / N)
print("RMSE is {}".format(rmse))
print("NRMSE is {}".format(rmse/mean_real_pow))

the mean is 109.18671703238813
r_2 is 0.8362628722386918
RMSE is 99.7252045341582
NRMSE is 0.9133455721044954


In [84]:
##OFFLINE 1
#the mean is 74.0599653418435
#r_2 is 0.6449911853863788
#RMSE is 95.8368082651351
#NRMSE is 1.2940433852861637
#[2758, 3828, 18875, 78, 30]


##ONLINE 1
#the mean is 74.0599653418435
#r_2 is 0.6002694823421173
#RMSE is 101.69425671452096
#NRMSE is 1.3731340035756705
#[4856, 13920, 6751, 34, 8]

# NOTE: only 8 new user

# old

In [71]:
'''
Predict the power consumption of a job.
    If the user and the job name are known, than it use the method 1 [user,job,nodes]
    if only the user is known, than it use the method 2 [user,nodes]
    Otherwise it use the method 3 [nodes]
'''

def predict_pow(user, job_name, used_nodes, used_cpus):
    pred_pow = 0.0
    
    methods = [0, 0, 0, 0, 0]
    
    # check if the method 1 or 2 is available
    mean_core_user_job = df_user_job_node.loc[(df_user_job_node['user']==user) & (df_user_job_node['job_name'] == job_name)]
    if(mean_core_user_job.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
        mean_core_wholeNodes = mean_core_user_job['mean_core'].mean()
        for node, ncpu in zip(used_nodes, used_cpus):
            mean_node = mean_core_user_job.loc[(mean_core_user_job['node_id']==node)]
            if(mean_node.shape[0] != 0):
                pred_pow += mean_node['mean_core'].iloc[0] * ncpu
                methods[0] += 1
            else:
                pred_pow += mean_core_wholeNodes * ncpu
                methods[1] += 1
        return (methods, pred_pow)
    
    
    # check if the method 3 or 4 is available
    mean_core_user = df_user_node.loc[(df_user_node['user']==user)]
    if(mean_core_user.shape[0] != 0):  # if != 0 than user exists on hystoric data
        mean_core_wholeUserJobs = mean_core_user['mean_core'].mean()
        for node, ncpu in zip(used_nodes, used_cpus):
            mean_node = mean_core_user.loc[(mean_core_user['node_id']==node)]
            if(mean_node.shape[0] != 0):
                pred_pow += mean_node['mean_core'].iloc[0] * ncpu
                methods[2] += 1
            else:
                pred_pow += mean_core_wholeUserJobs * ncpu
                methods[3] += 1
        return (methods, pred_pow)
    
    
    # method 5 is the only one available
    for node, ncpu in zip(used_nodes, used_cpus):
        mean_node = df_node.loc[(df_node['node_id']==node)]
        pred_pow += mean_node['mean_core'].iloc[0] * ncpu
        methods[4] += 1
    return (methods, pred_pow)

In [5]:
# Prediction

In [73]:
#debug
#merged_jobs_to_nodes[merged_jobs_to_nodes[['user', 'job_name', 'node_req', 'node_id', 'ncpus']]['node_req']>1]

In [ ]:
## OFFLINE 3 FILE

i = 0
jobs_test['predict_off_whole'] = 0.0
#jobs_test['method_off_whole'] = 0
global_methods = [0, 0, 0, 0, 0]

for index_job_test, row_job_test in jobs_test.iterrows():
    merged_job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == row_job_test['job_id_string']]
    grouped_job_to_nodes = merged_job_to_nodes.groupby(['node_id', 'ncpus','ngpus', 'nmics']).size().reset_index(name='counts')
    
    used_nodes = grouped_job_to_nodes['node_id']
    used_cpus = grouped_job_to_nodes['ncpus']
    
    methods, predicted_pow = predict_pow(row_job_test['user'], row_job_test['job_name'], used_nodes, used_cpus)
    
    for j in range(5):
        global_methods[j] += methods[j]
    
    #global_methods[0] += methods[0]
    #global_methods[1] += methods[1]
    #global_methods[2] += methods[2]
    #global_methods[3] += methods[3]
    #global_methods[4] += methods[4]
    
       
    #print("{}: {} ------- {}".format(method, predicted_pow, row_job_test['real_pow']))
    jobs_test.at[index_job_test, 'predict_off_whole'] = predicted_pow
    #jobs_test.at[index_job_test, 'method_off_whole'] = method
    
    i += 1
    if(i%1000 == 0):
        print("{}/{}".format(i, jobs_test.shape[0]))
    #print("{}/{}".format(i, jobs_test.shape[0]))

In [44]:
global_methods

[2758, 3828, 0, 18953, 30]

In [46]:
merged_jobs_to_nodes.shape[0]

25569

In [45]:
2758+3828+18953+30

25569

In [ ]:
# method 1 whole
#the mean is 104.10646370851616
#r_2 is 0.7946394725232087
#RMSE is 128.29866772525662
#NRMSE is 1.2323794618984976

# method 1
#the mean is 146.6539413363719
#r_2 is 0.7891392216999262
#RMSE is 165.15338225831218
#NRMSE is 1.1261434964063404

# all
#the mean is 74.0599653418435
#r_2 is 0.6330236798554962
#RMSE is 97.43877023494733
#NRMSE is 1.3156739918145077



# user jobs
#the mean is 110.35204538702968
#r_2 is 0.810136180971846
#RMSE is 117.51841984031236
#NRMSE is 1.0649410206049972

In [5]:
infile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node"
df_user_job_node = pd.read_csv(infile_user_job_node + ".csv", index_col=0)
infile_user_job = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job"
df_user_job = pd.read_csv(infile_user_job + ".csv", index_col=0)
infile_user_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_node"
df_user_node = pd.read_csv(infile_user_node + ".csv", index_col=0)
infile_user = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user"
df_user = pd.read_csv(infile_user + ".csv", index_col=0)
infile_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_node"
df_node = pd.read_csv(infile_node + ".csv", index_col=0)